In [1]:
# !pip install openai
# !pip install playwright
# pip install --upgrade openai

In [2]:
import openai
import os
import subprocess
from PIL import Image
from openai import OpenAIError
from dotenv import load_dotenv

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

TRANSCRIPT_FILE = "transcript-ds.txt"
PROMPT_FILE = "prompt-ds.txt"

# Output Files
CODE_JS_FILE = "sketch.js"
IMAGE_FILE = "img.png"
HTML_FILE = "index.html"
P5_JS_LIBRARY = "p5.js"

In [4]:
client = openai.Client(
    api_key=OPENAI_API_KEY
)

try:
    with open(PROMPT_FILE, "r", encoding="utf-8") as f:
        gpt_prompt = f.read()
    with open(TRANSCRIPT_FILE, "r", encoding="utf-8") as f:
        transcript_content = f.read()
except FileNotFoundError as e:
    print(f"Error: File not found: {e.filename}")
    exit(1)
except Exception as e:
    print(f"Error reading file: {e}")
    exit(1)

try:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": gpt_prompt},
            {"role": "user", "content": f"<transcript>{transcript_content}<transcript>"}
        ],
        temperature=0.2,  # Adjust for creativity
    )
    code_js = response.choices[0].message.content
except OpenAIError as e:
    print(f"OpenAI API Error: {e}")
    exit(1)
except Exception as e:
    print(f"Error during GPT interaction: {e}")
    exit(1)

In [5]:
code_js

'```javascript\nlet nodes = [\'A\', \'B\', \'C\', \'D\', \'E\'];\nlet positions = [];\nlet radius = 20;\nlet spacing = 100;\n\nfunction setup() {\n  createCanvas(600, 200);\n  for (let i = 0; i < nodes.length; i++) {\n    positions.push(createVector(50 + i * spacing, height / 2));\n  }\n  setTimeout(() => { saveCanvas(\'img\', \'png\'); document.body.setAttribute("data-ready", "true"); }, 500);\n}\n\nfunction draw() {\n  background(255);\n  stroke(0);\n  fill(255);\n\n  for (let i = 0; i < positions.length; i++) {\n    ellipse(positions[i].x, positions[i].y, radius * 2, radius * 2);\n    fill(0);\n    textAlign(CENTER, CENTER);\n    text(nodes[i], positions[i].x, positions[i].y);\n    fill(255);\n    if (i < positions.length - 1) {\n      line(positions[i].x + radius, positions[i].y, positions[i + 1].x - radius, positions[i + 1].y);\n    }\n  }\n}\n```'

In [6]:
# --- Save p5.js Code ---
import re
cleaned_code = re.sub(r"```javascript\n|```", "", code_js)

try:
    with open(CODE_JS_FILE, "w", encoding="utf-8") as f:
        f.write(cleaned_code)
    print(f"Generated p5.js code saved to: {CODE_JS_FILE}")
except Exception as e:
    print(f"Error saving p5.js code: {e}")
    exit(1)


Generated p5.js code saved to: sketch.js


In [7]:
# import asyncio
# from playwright.async_api import async_playwright

# async def capture_screenshot():
#     async with async_playwright() as p:
#         browser = await p.chromium.launch()
#         page = await browser.new_page()

#         # Open the p5.js sketch URL (Replace with your actual URL)
#         await page.goto("http://localhost:8000")

#         # Wait for the canvas to be ready
#         await page.wait_for_selector("canvas")

#         # Take a screenshot and save it as "screenshot.png"
#         await page.screenshot(path="img.png")

#         await browser.close()
#         print("Diagram saved as 'img.png'")

# # Run the function inside Jupyter Notebook
# await capture_screenshot()

In [8]:
import asyncio
from playwright.async_api import async_playwright

async def capture_image():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()

        # Open the local p5.js sketch
        await page.goto("http://localhost:8000/index.html")

        # Wait for the body to get the 'data-ready' attribute
        await page.wait_for_selector("body[data-ready='true']")

        # Capture the image
        await page.screenshot(path="img.png")

        await browser.close()
        print("Image saved as img.png")

# Run the async function in Jupyter Notebook
await capture_image()

Image saved as img.png
